In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Hypothyroidism/GSE18152'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Screening for chromosomal aberrations by array CGH in 74 patients with congenital hypothyroidism"
!Series_summary	"Congenital Hypothyroidism occurs in 1:3500 live births and is therefore the most common congenital endocrine disorder. A spectrum of defective thyroid morphology, termed thyroid dysgenesis, represents 80% of permanent CH cases. Although several candidate genes have been implicated in thyroid development, comprehensive screens failed to detect mutation carriers in a significant number of patients with non-syndromic TD. Due to the sporadic occurrence of TD, de novo chromosomal rearrangements are conceivably representing one of the molecular mechanisms participating in its aetiology. Recently, the use of array CGH technique has provided the ability to map these variations genomewide with high resolution. We performed an array CGH screen of 74 TD patients to determine the role of copy number variants (CNV) in the aetiology of the disease.

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Ensure the sample_characteristics dictionary is available
sample_characteristics = {
    0: ['tissue: blood of a patient with ch.', 'tissue: blood of a healthy patient.', 'tissue: blood of a patient with ch', 'tissue: blood of a patient with ch (athyreosis)', 'tissue: blood of a patient with ch (hypoplasia)', 'tissue: blood of a patient with ch (discordant twin to 1302)', 'tissue: blood of a patient with ch (athreosis)'],
    1: ['gender: male', 'gender: female']
}

# Step 1: Check if the dataset contains gene expression data
if "tissue: blood of a patient with ch" in sample_characteristics[0] or \
   "tissue: blood of a healthy patient." in sample_characteristics[0]:
    is_gene_available = True

# Step 2: Variables Availability and Data Type Conversion
# Hypothyroidism (Trait)
if any("tissue: blood of a patient" in item for item in sample_characteristics[0]):
    trait_row = 0

# Gender
if "gender: male" in sample_characteristics[1] and "gender: female" in sample_characteristics[1]:
    gender_row = 1

# Define conversion functions
def convert_trait(value):
    if value.startswith("tissue:"):
        val = value.split(':')[1].strip()
        if "healthy" in val:
            return 0
        elif "patient" in val:
            return 1
    return None

def convert_gender(value):
    if value.startswith("gender:"):
        val = value.split(':')[1].strip()
        if val == "male":
            return 1
        elif val == "female":
            return 0
    return None

# Placeholder for convert_age if age data is found later
def convert_age(value):
    return None  # Update with actual logic if age is available in future steps

# Step 3: Save Metadata
save_cohort_info('GSE18152', './preprocessed/Hypothyroidism/cohort_info.json', is_gene_available, trait_row is not None)

# Step 4: Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Hypothyroidism', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Hypothyroidism/trait_data/GSE18152.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM453685': [1, 1], 'GSM453686': [1, 1], 'GSM453687': [1, 1], 'GSM453688': [1, 0], 'GSM453689': [1, 0], 'GSM453690': [1, 0], 'GSM453691': [1, 0], 'GSM453692': [1, 0], 'GSM453693': [1, 0], 'GSM453694': [1, 0], 'GSM453695': [1, 0], 'GSM453696': [1, 0], 'GSM453697': [1, 0], 'GSM453698': [1, 0], 'GSM453699': [1, 0], 'GSM453700': [1, 0], 'GSM453701': [1, 0], 'GSM453702': [1, 0], 'GSM453703': [1, 0], 'GSM453704': [1, 1], 'GSM453705': [1, 1], 'GSM453706': [1, 1], 'GSM453707': [1, 1], 'GSM453708': [1, 1], 'GSM453709': [1, 0], 'GSM453710': [1, 0], 'GSM453711': [1, 0], 'GSM453712': [1, 0], 'GSM453713': [1, 0], 'GSM453714': [1, 0], 'GSM453715': [1, 0], 'GSM453716': [1, 0], 'GSM453717': [1, 0], 'GSM453718': [1, 0], 'GSM453719': [1, 1], 'GSM453721': [0, 0], 'GSM453722': [0, 0], 'GSM453723': [0, 0], 'GSM453724': [0, 0], 'GSM453725': [0, 0], 'GSM453726': [0, 1], 'GSM453735': [1, 1], 'GSM453736': [1, 1], 'GSM453737': [1, 1], 'GSM453738': [1, 0], 'GSM453739': [1, 0], 'GSM453740': [1, 0], 'GSM453741':

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['10_10_11_N0010J19', '10_10_17_N0014K03', '10_10_19_N0012J05',
       '10_10_1_dJ1118A7', '10_10_20_N0335K08', '10_10_6_M2532O10',
       '10_11_10_N0692C05', '10_11_16_N0674B13', '10_11_17_N0081L05',
       '10_12_11_N0026F24', '10_12_12_N0280F22', '10_12_13_N0206M11',
       '10_12_14_N0012J09', '10_12_16_N0523B21', '10_12_2_RP11148f10',
       '10_12_3_L05299', '10_13_13_N0186B11', '10_13_15_N0044A06',
       '10_13_1_dJ605G11', '10_13_2_RP1188n2'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1_1_1_bK1444N24', '1_1_2_Buffer_PCR', '1_1_3_397B22', '1_1_4_Buffer_PCR', '1_1_5_Buffer_PCR'], 'Internal_Name': ['bK1444N24', 'Buffer_PCR', '397B22', 'Buffer_PCR', 'Buffer_PCR'], 'Name': ['bK1444N24', 'Buffer_PCR', '397B22', 'Buffer_PCR', 'Buffer_PCR'], 'Block': ['1', '1', '1', '1', '1'], 'Row': ['1', '1', '1', '1', '1'], 'Col': ['1', '2', '3', '4', '5'], 'Chromosom': ['--', '--', '--', '--', '--'], 'Chromosom_start': ['--', '--', '--', '--', '--'], 'Chromosom_end': ['--', '--', '--', '--', '--'], 'GB_RANGE': [nan, nan, nan, nan, nan], 'CLONE_ID': [nan, nan, nan, nan, nan], 'SPOT_ID': ['bK1444N24', '--Buffer_PCR', '397B22', '--Buffer_PCR', '--Buffer_PCR']}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the keys for mapping gene identifiers to gene symbols
identifier_key = 'ID'
gene_symbol_key = 'Internal_Name'

# 2. Get the dataframe storing the mapping between probe IDs and genes
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping to create the gene expression dataframe
gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Hypothyroidism/gene_data/GSE18152.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Hypothyroidism')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE18152', './preprocessed/Hypothyroidism/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Hypothyroidism/GSE18152.csv'
    unbiased_merged_data.to_csv(csv_path)


No gene data in the dataframe
